In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
from pymongo import MongoClient 

# Correct the import of the AnimalShelter class from your Milestone 6 file
from animal_Shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Update with your username and password and CRUD Python module name.
username = "aacuser"     # Your MongoDB username
password = "Alice2018"   # Your MongoDB password
shelter = AnimalShelter(username, password)  # Instantiate the CRUD class

# Read the data from MongoDB using the 'read' method and load into a DataFrame
df = pd.DataFrame.from_records(shelter.read({}))

# Remove '_id' column if it exists to avoid issues with Dash DataTable
if '_id' in df.columns:
    df = df.drop('_id', axis=1)

# Debugging: Check if the DataFrame is empty
if df.empty:
    print("No data retrieved from MongoDB.")
else:
    print("Data retrieved:", df.head())

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Milestone6Dashboard')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Rafael CS-340 Dashboard'))), 
    html.Hr(),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    
    # Map display
    html.Div(id='map-id', className='col s12 m6')
])

#############################################
# Interaction Between Components / Controller
#############################################
# Callback to highlight selected row in DataTable
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_table_highlight(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback to update the map based on selected row in DataTable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_map(viewData):
    if viewData is None or len(viewData) == 0:
        return []  # Return empty if no data is available

    dff = pd.DataFrame.from_dict(viewData)

    if dff.empty:
        return []  # Return empty if DataFrame is empty

    # Assuming 'location_lat' and 'location_long' are the correct field names
    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[dff.loc[0, 'location_lat'], dff.loc[0, 'location_long']], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.loc[0, 'location_lat'], dff.loc[0, 'location_long']], children=[
                dl.Tooltip(dff.iloc[0]['name']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0, 'name'])
                ])
            ])
        ])
    ]

# Run the app server
app.run_server(debug=True, port=31162, host='127.0.0.1')




Data retrieved:    rec_num age_upon_outcome animal_id animal_type                     breed  \
0      1.0          3 years   A746874         Cat    Domestic Shorthair Mix   
1      2.0           1 year   A725717         Cat    Domestic Shorthair Mix   
2      3.0          2 years   A716330         Dog   Chihuahua Shorthair Mix   
3      5.0          2 years   A691584         Dog    Labrador Retriever Mix   
4      6.0          5 years   A696004         Dog  Cardigan Welsh Corgi Mix   

          color date_of_birth             datetime            monthyear  \
0   Black/White    2014-04-10  2017-04-11 09:00:00  2017-04-11T09:00:00   
1  Silver Tabby    2015-05-02  2016-05-06 10:49:00  2016-05-06T10:49:00   
2   Brown/White    2013-11-18  2015-12-28 18:43:00  2015-12-28T18:43:00   
3     Tan/White    2012-11-06  2015-05-30 13:48:00  2015-05-30T13:48:00   
4   Sable/White    2010-01-27  2015-01-28 10:39:00  2015-01-28T10:39:00   

    name outcome_subtype     outcome_type sex_upon_outcome